<a href="https://colab.research.google.com/github/iffan17/FRA333_HW3_6553_6563/blob/main/FRA503_HW2_6546_6563.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FRA502 Deep Reinforcement Learning For Robotics
## ภนลภัส สุทธิมาลา 65340500046
## อิฟฟาน อัครามีน 65340500063

In [ ]:
# 📊 "Data Analysis Module" (Sneaky Number Guesser)
import random

def fake_data_report():
    print("📈 Calculating metrics...")
    secret = random.randint(1, 100)
    tries = 0

    while True:
        guess = input("Enter dataset ID (1-100): ").strip()
        if not guess.isdigit():
            print("⚠️ Invalid input format.")
            continue
        guess = int(guess)
        tries += 1

        if guess < secret:
            print("➡️ Higher variance detected!")
        elif guess > secret:
            print("⬅️ Lower trend observed!")
        else:
            print(f"✅ Anomaly found in {tries} attempts!")
            break

fake_data_report()

📈 Calculating metrics...
Enter dataset ID (1-100): 99
⬅️ Lower trend observed!
Enter dataset ID (1-100): 23
➡️ Higher variance detected!
Enter dataset ID (1-100): 58
➡️ Higher variance detected!
Enter dataset ID (1-100): 69
➡️ Higher variance detected!
Enter dataset ID (1-100): 89
⬅️ Lower trend observed!
Enter dataset ID (1-100): 70
➡️ Higher variance detected!
Enter dataset ID (1-100): 75
⬅️ Lower trend observed!
Enter dataset ID (1-100): 73
✅ Anomaly found in 8 attempts!


# **Part 1: Setting up `Cart-Pole` Agent**

## **1. RL Base Class**

ในส่วนนี้ เราจะสร้าง Base Class หลักของ Reinforcement Learning (RL) สำหรับ Cart-Pole ซึ่งจะมีการกำหนด **Constructor** และ **Core Functions** ตามรายละเอียดในข้อกำหนด

### **Constructor**

Constructor จะมีพารามิเตอร์สำคัญ เช่น:
- **Control type**: กำหนดว่าเป็น RL Algorithm แบบใด (เช่น Q-Learning, SARSA, Double Q-Learning)
- **Number of actions**: จำนวน discrete actions ที่ agent สามารถเลือกได้
- **Action range**: ช่วงค่าต่ำสุด (min) และสูงสุด (max) ของ action (continuous)
- **Discretize state weight**: ใช้กำหนดการ discretize state (กำหนด bin สำหรับแต่ละตัวแปร state)
- **Learning rate**: ควบคุมความเร็วในการอัปเดต Q-Value
- **Initial epsilon**: ค่าเริ่มต้นของ epsilon ใน ε-greedy policy
- **Epsilon decay rate**: อัตราการลดลงของ epsilon เมื่อเวลาผ่านไป
- **Final epsilon**: ค่า epsilon ต่ำสุด ที่จะยังมีโอกาส exploration
- **Discount factor (γ)**: กำหนดความสำคัญของรางวัลในอนาคต

### **Core Functions**

1. `get_discretize_action()`: คืนค่า discrete action จาก policy โดยใช้ ε-greedy
2. `mapping_action()`: แปลง discrete action ไปสู่ค่า continuous ภายใน `[action_min, action_max]`
3. `discretize_state()`: จัดแบ่ง (discretize) และ scale ค่า state ที่เป็น continuous ให้อยู่ในรูป discrete เพื่อให้ agent ใช้ได้ง่าย
4. `decay_epsilon()`: ลดค่า epsilon ตามอัตราการลดที่กำหนด
5. `save_q_value()` และ `load_q_value()`: บันทึก/โหลด Q(s,a) ที่เก็บในลักษณะ dictionary หรือ defaultdict

จาก Cart-Pole environment ข้อมูล state ที่ได้มี 4 components หลัก ๆ ได้แก่:
1. **Relative joint velocity** ของ cart
2. **Relative joint velocity** ของ pole
3. **Relative joint position** ของ cart
4. **Relative joint position** ของ pole

แต่ละ component เป็นค่า continuous และช่วงค่าอาจกว้างมาก ดังนั้นจำเป็นต้อง discretize ให้เป็นช่วง (bins) เพื่อทำงานร่วมกับตาราง Q-Value ได้สะดวก

ตัวอย่างการกำหนดขอบเขต (boundary) คร่าว ๆ ที่ใช้ในการ clip:
1. **cart position**: [-4.5, 4.5]
2. **pole angle**: [-π, π]
3. **cart velocity**: [-10, 10]
4. **pole angular velocity**: [-10, 10]

#### **ตัวอย่างโค้ดฟังก์ชัน `discretize_state(self, obs)`**

In [ ]:
def discretize_state(self, obs: dict):
    # กำหนดจำนวน bins สำหรับแต่ละตัวแปร
    pose_cart_bin = self.discretize_state_weight[0]
    pose_pole_bin = self.discretize_state_weight[1]
    vel_cart_bin = self.discretize_state_weight[2]
    vel_pole_bin = self.discretize_state_weight[3]

    # ขอบเขตสำหรับ clipping
    pose_cart_bound = 4.5
    pose_pole_bound = np.pi
    vel_cart_bound = 10
    vel_pole_bound = 10

    # ดึงค่าจาก obs['policy'] (เป็นตัวอย่างการอ่าน)
    pose_cart_raw = obs['policy'][0, 0]
    pose_pole_raw = obs['policy'][0, 1]
    vel_cart_raw  = obs['policy'][0, 2]
    vel_pole_raw  = obs['policy'][0, 3]

    # clip ค่าไม่ให้ออกจากขอบเขต
    pose_cart_clip = torch.clip(pose_cart_raw, -pose_cart_bound, pose_cart_bound)
    pose_pole_clip = torch.clip(pose_pole_raw, -pose_pole_bound, pose_pole_bound)
    vel_cart_clip  = torch.clip(vel_cart_raw, -vel_cart_bound, vel_cart_bound)
    vel_pole_clip  = torch.clip(vel_pole_raw, -vel_pole_bound, vel_pole_bound)

    device = pose_cart_clip.device

    # สร้าง grid แบบ linspace สำหรับแต่ละตัวแปร
    pose_cart_grid = torch.linspace(-pose_cart_bound, pose_cart_bound, pose_cart_bin, device=device)
    pose_pole_grid = torch.linspace(-pose_pole_bound, pose_pole_bound, pose_pole_bin, device=device)
    vel_cart_grid  = torch.linspace(-vel_cart_bound, vel_cart_bound, vel_cart_bin, device=device)
    vel_pole_grid  = torch.linspace(-vel_pole_bound, vel_pole_bound, vel_pole_bin, device=device)

    # bucketize เพื่อเปลี่ยนค่าจาก continuous -> discrete
    pose_cart_dig = torch.bucketize(pose_cart_clip, pose_cart_grid)
    pose_pole_dig = torch.bucketize(pose_pole_clip, pose_pole_grid)
    vel_cart_dig  = torch.bucketize(vel_cart_clip,  vel_cart_grid)
    vel_pose_dig  = torch.bucketize(vel_pole_clip,  vel_pole_grid)

    return (
        int(pose_cart_dig),
        int(pose_pole_dig),
        int(vel_cart_dig),
        int(vel_pose_dig)
    )

**สรุปการทำงาน:**
- คลิปค่าไม่ให้ออกนอกขอบเขต
- สร้าง grid ด้วย `torch.linspace()` เพื่อแบ่งช่วงเป็น bins
- ใช้ `torch.bucketize()` เพื่อแปลง continuous value ไปเป็น discrete index
- คืนค่าเป็น tuple สำหรับ state (เช่น `(idx_cart_pose, idx_pole_pose, idx_cart_vel, idx_pole_vel)`) เพื่อนำไปอัปเดตหรือค้นหา Q-Value

#### **ตัวอย่างโค้ดฟังก์ชัน `get_discretize_action(self, obs_dis) -> int:`**

เลือก action โดยใช้ ε-greedy policy:
- ถ้าสุ่ม `rand` <= epsilon -> เลือก action แบบสุ่ม (exploration)
- ถ้า `rand` > epsilon -> เลือก action ที่ให้ค่าตอบแทนสูงสุดจากตาราง Q (exploitation)

In [ ]:
def get_discretize_action(self, obs_dis) -> int:
    rand = np.random.rand()
    if rand <= self.epsilon:
        # Exploration: เลือก action แบบสุ่ม
        return np.random.randint(0, self.num_of_action)
    else:
        # Exploitation: เลือก action ที่มีค่ามากสุดใน q_values
        return np.argmax(self.q_values[obs_dis])

#### **ตัวอย่างโค้ดฟังก์ชัน `mapping_action(self, action) -> torch.Tensor:`**

แปลง discrete action ไปเป็น continuous value ภายใน `[action_min, action_max]`

In [ ]:
def mapping_action(self, action_idx):
    """
    เปลี่ยนค่า action แบบ discrete [0..(n-1)] ไปเป็นค่าต่อเนื่อง [min, max]
    """
    step_size = (self.action_range[1] - self.action_range[0]) / (self.num_of_action - 1)
    continuous_value = self.action_range[0] + action_idx * step_size
    return torch.tensor([[continuous_value]])  # shape ตัวอย่าง (1,1)

#### **ตัวอย่างโค้ดฟังก์ชัน `decay_epsilon(self):`**

ลดค่าของ epsilon ในแต่ละ episode เพื่อให้ agent ค่อย ๆ ลดการสุ่มสำรวจ

In [ ]:
def decay_epsilon(self):
    self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)

หากใช้ decay รูปอื่น (เช่น exponential) ก็ปรับ logic ได้ตามต้องการ

#### **save_q_value()** และ **load_q_value()**
ใช้บันทึกค่าตาราง Q(s,a) เพื่อใช้งานภายหลังหรือทดสอบแบบ offline

---
## **2. Learning Algorithms**

ในโฟลเดอร์ **Algorithm** แต่ละคลาสจะสืบทอดจาก Base Class และมีฟังก์ชัน `update()` เป็นแกนหลักในการอัปเดต Q-Value หรือข้อมูลที่จำเป็น

### **Monte Carlo**
- วิธีการอัปเดตค่า Q-value เมื่อจบ episode (First-visit หรือ Every-visit)
- สะสม reward กลับย้อนหลัง (return G) และอัปเดตเฉพาะ state-action ที่เพิ่งเจอครั้งแรกใน episode นั้น

In [ ]:
def update(self, obs_dis, action_idx, reward, done):
    self.obs_hist.append(obs_dis)
    self.action_hist.append(action_idx)
    self.reward_hist.append(reward)

    if done:
        G_cum = 0  # return สะสมจากท้าย episode
        for t in reversed(range(len(self.obs_hist))):
            G_cum = self.discount_factor * G_cum + self.reward_hist[t]
            # First-visit checking
            if (self.obs_hist[t], self.action_hist[t]) not in list(zip(self.obs_hist[:t], self.action_hist[:t])):
                self.n_values[self.obs_hist[t]][self.action_hist[t]] += 1
                old_q = self.q_values[self.obs_hist[t]][self.action_hist[t]]
                self.q_values[self.obs_hist[t]][self.action_hist[t]] = old_q + (G_cum - old_q) / \
                    self.n_values[self.obs_hist[t]][self.action_hist[t]]

        self.obs_hist.clear()
        self.action_hist.clear()
        self.reward_hist.clear()

### **SARSA**
- อัปเดตค่า Q ด้วยรูปแบบ on-policy
- ใช้ค่าของ `Q(s_{t+1}, a_{t+1})` ที่เลือกจาก policy เดียวกัน (ε-greedy) มา bootstrap

$$Q(S_t,A_t) \leftarrow Q(S_t,A_t) + \alpha [R_{t+1} + \gamma \cdot Q(S_{t+1}, A_{t+1}) - Q(S_t,A_t)]$$

In [ ]:
def update(self, obs_dis, action_idx, reward, next_obs_dis, done):
    if done:
        # หากจบ episode ไม่มี next state ให้ bootstrap
        self.q_values[obs_dis][action_idx] += self.lr * (reward - self.q_values[obs_dis][action_idx])
    else:
        # เลือก action สำหรับ state ถัดไป ด้วย epsilon-greedy แบบ SARSA
        next_a = self.get_discretize_action(next_obs_dis)
        td_target = reward + self.discount_factor * self.q_values[next_obs_dis][next_a]
        td_error = td_target - self.q_values[obs_dis][action_idx]
        self.q_values[obs_dis][action_idx] += self.lr * td_error

### **Q-Learning**
- Off-policy: ใช้ max action value ในการ bootstrap แทนที่จะเป็น action ที่เลือกได้จริง

$$Q(S_t,A_t) \leftarrow Q(S_t,A_t) + \alpha [R_{t+1} + \gamma \cdot \max_{a} Q(S_{t+1}, a) - Q(S_t,A_t)]$$

In [ ]:
def update(self, obs_dis, action_idx, next_obs_dis, reward):
    best_next_action = np.max(self.q_values[next_obs_dis])
    td_target = reward + self.discount_factor * best_next_action
    td_error = td_target - self.q_values[obs_dis][action_idx]
    self.q_values[obs_dis][action_idx] += self.lr * td_error

### **Double Q-Learning**
- ใช้ Q-table สองตัว (QA และ QB) สลับกันอัปเดตเพื่อบรรเทาการ overestimation ของ Q-Learning
- เลือก action จาก QA แต่ประเมินด้วย QB หรือเลือก action จาก QB แต่ประเมินด้วย QA

In [ ]:
def update(self, obs_dis, action_idx, next_obs_dis, reward):
    if np.random.rand() < 0.5:
        a_star = np.argmax(self.qa_values[next_obs_dis])
        td_target = reward + self.discount_factor * self.qb_values[next_obs_dis][a_star]
        td_error = td_target - self.qa_values[obs_dis][action_idx]
        self.qa_values[obs_dis][action_idx] += self.lr * td_error
    else:
        b_star = np.argmax(self.qb_values[next_obs_dis])
        td_target = reward + self.discount_factor * self.qa_values[next_obs_dis][b_star]
        td_error = td_target - self.qb_values[obs_dis][action_idx]
        self.qb_values[obs_dis][action_idx] += self.lr * td_error

    # ปรับให้ q_values หลักเท่ากับผลรวมจาก QA/QB อย่างใดอย่างหนึ่ง
    self.q_values[obs_dis][action_idx] = self.qa_values[obs_dis][action_idx]

โดยแนวคิดคือ ให้ QA และ QB แบ่งหน้าที่กันคนละรอบ จึงช่วยลดการประเมินค่าสูงเกินจริง (overestimation) ที่มักเกิดใน Q-Learning

---

# **Part 2: Training & Tuning to Stabilize Cart-Pole Agent**

## ใน Part นี้ เราทำการเทรน และปรับจูนโมเดลทั้ง 4 Model เพื่อหา Hyperparameter ที่เหมาะสม

1.1 โดยเราเริ่มจากการปรับจูน hyperparameter ของ Q-Learning ซึ่งเป็น Base model
เนื่องจาก Q-Learning ใช้เวลาในการเทรนน้อยกว่ารุ่นอื่น และสามารถสังเกตการเรียนรู้ได้ชัดเจน

1.2 หลังจากได้ชุด parameter ที่ให้ performance ดีที่สุดจาก Q-Learning แล้ว
เราจึงนำชุด parameter เดิมไปทดลองกับ DQL, SARSA และ MC เพื่อเปรียบเทียบว่า
แต่ละโมเดลมีความสามารถในการเรียนรู้แตกต่างกันหรือไม่ ภายใต้เงื่อนไขเดียวกัน

1.3 นอกจากนี้ เรายังทำการทดลองเพิ่ม-ลดค่าของ hyperparameter ที่สำคัญบางตัว
เพื่อดูว่าโมเดลต่างๆ มีความไว (sensitive) ต่อการเปลี่ยนแปลง parameter อย่างไรบ้าง

แน่นอน! ด้านล่างคือคำอธิบายของ **hyperparameters** ที่ใช้อยู่ พร้อมบอกว่าใน **Part 2** ของรายงาน (Training & Tuning) เราปรับตัวไหนบ้าง — ในรูปแบบ Bullet Point พร้อมใช้ใน Jupyter Notebook:

---

## 🔧 Hyperparameter Overview (Used in Part 2)

- `num_of_action`
  - ความละเอียดของ action space (จำนวน action ที่สามารถเลือกได้)
  - ปรับใน **Run 3 (Action/Observation space test)** เพื่อดูผลต่อ learning efficiency

- `action_range`
  - ขอบเขตค่าของ action (เช่น แรงที่ใช้ในการควบคุมรถเข็น)
  - **ไม่ปรับในรอบนี้** (ใช้ค่าคงที่จาก base parameter)

- `discretize_state_weight`
  - ความละเอียดของการแปลง observation (continuous state) ให้เป็น discrete state
  - ปรับใน **Run 3 (Action/Observation space test)** คู่กับ `num_of_action`

- `learning_rate`
  - ควบคุมความเร็วในการอัปเดต Q-values
  - ปรับใน **Run 1 (Min LR)** และ **Run 2 (Max LR)** เพื่อหา learning efficiency ที่ดีที่สุด

- `n_episodes`
  - จำนวนรอบการฝึกในแต่ละ run
  - ใช้ `5,000` สำหรับ tuning ทุก run  
  - ใช้ `20,000` เฉพาะสำหรับ final run ของแต่ละโมเดล

- `start_epsilon`
  - ค่าเริ่มต้นของ epsilon ใน epsilon-greedy policy (ใช้ควบคุมการ explore)
  - ใช้ค่าคงที่ `1.0` ตลอดทุกการทดลอง

- `epsilon_decay`
  - อัตราการลดลงของ epsilon ต่อ episode
  - ปรับใน **Run 4 (Min ED)** และ **Run 5 (Max ED)**

- `final_epsilon`
  - ค่าต่ำสุดของ epsilon (ขั้นต่ำของการ explore)
  - **ไม่ปรับในรอบนี้** (คงไว้ที่ `0.05`)

- `discount`
  - discount factor สำหรับ future reward
  - **ไม่ปรับในรอบนี้** (คงไว้ที่ `1`)

---

หากต้องการต่อใน cell ถัดไป ก็สามารถแยกใส่การทดลองของแต่ละ run ต่อได้เลยครับ.

In [ ]:
# Parameter สำหรับการ Train Agent
num_of_action = 5
action_range = [-12, 12]  # ช่วงของ action
discretize_state_weight = [4, 8, 4, 4]  # จำนวน bins สำหรับแต่ละ state: [pose_cart, pose_pole, vel_cart, vel_pole]
learning_rate = 0.03
n_episodes = 5000
start_epsilon = 1.0
epsilon_decay = 0.00003  # ลดความน่าจะเป็นในการสำรวจ
final_epsilon = 0.05
discount = 1

# Parameter สำหรับ clipping ของ state values
pose_cart_bound = 3
pose_pole_bound = float(np.deg2rad(24.0))
vel_cart_bound = 15
vel_pole_bound = 15

#### **การกำหนดขนาดของ State Space**

เพื่อให้ agent สามารถเยี่ยมชมทุก state ที่เป็นไปได้ เราเลือกที่จะกำหนด state space ให้มีขนาดเล็ก โดยคำนวณได้ดังนี้:

$$N_S = S_{pose\_cart} \times S_{pose\_pole} \times S_{vel\_cart} \times S_{vel\_pole}$$

โดยที่จำนวน state (bins) แต่ละตัวถูกกำหนดโดย `discretize_state_weight` เช่น ในที่นี้:

$$N_S = 4 \times 8 \times 4 \times 4 = 512$$

และขนาดของ Q-table คือ:

$$N_Q = N_S \times N_A = 512 \times 5 = 2560$$

ซึ่งหมายความว่าในแต่ละ episode agent มีโอกาสเยี่ยมชม state-action pair ได้บ่อยครั้ง

#### **การ Train & การ Play**

ในขั้นตอนนี้ เราจะทำการฝึก (Train) Agent ด้วยคำสั่งใน terminal และใช้ script สำหรับการทดสอบ (Play) โดยใช้คำสั่งดังนี้:

```bash
# ฝึก Agent สำหรับ Stabilizing Cart-Pole Task
python scripts/RL_Algorithm/train.py --task Stabilize-Isaac-Cartpole-v0

# ทดสอบ Agent หลังจากที่ฝึกเสร็จ
python scripts/RL_Algorithm/play.py --task Stabilize-Isaac-Cartpole-v0
```

หมายเหตุ: ในที่นี้เราจะทำเฉพาะการทดลอง Stabilizing Cart-Pole Task โดยไม่รวม Swing-up

### **Experiment Runs Parameters Summary**

---

###  **1. QL Tuning (5K Ep. Each)**  
#### **Run 1:** 5K QL - Min LR & Min Epsilon Decay
```python
learning_rate = 0.01
epsilon_decay = 0.00001
n_episodes = 5000
```

#### **Run 2:** 5K QL - Max LR & Max Epsilon Decay
```python
learning_rate = 0.1
epsilon_decay = 0.0001
n_episodes = 5000
```

#### **Run 3:** 5K QL - Action/Obs Space Test **ไม่ได้บันทึกผลไว้
```python
num_of_action = 3
discretize_state_weight = [3, 6, 3, 3]
n_episodes = 5000
```



### **2. Model Comparison (5K Ep. Each)**  

#### **Run 4:** 5K QL - Final/Base
```python
learning_rate = 0.03
epsilon_decay = 0.00003
num_of_action = 5
discretize_state_weight = [4, 8, 4, 4]
n_episodes = 5000
```

---

#### **Run 5:** 5K DQL - Final/Base  
```python
learning_rate = 0.03
epsilon_decay = 0.00003
num_of_action = 5
discretize_state_weight = [4, 8, 4, 4]
n_episodes = 5000
```



#### **Run 7:** 5K MC - Final/Base   
```python
learning_rate = 0.03
epsilon_decay = 0.00003
num_of_action = 5
discretize_state_weight = [4, 8, 4, 4]
n_episodes = 5000
```

#### **Run 8:** 5K SARSA - Final/Base
```python
learning_rate = 0.03
epsilon_decay = 0.00003
num_of_action = 5
discretize_state_weight = [4, 8, 4, 4]
n_episodes = 5000
```


#### **Run 6:** 5K DQL - Lower LR
```python
learning_rate = 0.01
epsilon_decay = 0.00003
num_of_action = 5
discretize_state_weight = [4, 8, 4, 4]
n_episodes = 5000
```

#### **Run 9:** 5K SARSA - Lower Epsilon Decay   
```python
learning_rate = 0.03
epsilon_decay = 0.00001
num_of_action = 5
discretize_state_weight = [4, 8, 4, 4]
n_episodes = 5000
```

--

#### **สรุปผลการ Train**

ผลลัพธ์ที่ได้จะรวมถึงการเปลี่ยนแปลงของ Reward ต่อ Episode และ Q-Value Distribution ซึ่งสามารถ plot ได้ในขั้นตอนการ Evaluate (Part 3)

นอกจากนี้ คุณสามารถปรับพารามิเตอร์และเปรียบเทียบประสิทธิภาพของโมเดลต่าง ๆ โดยมีแผนการทดลองดังนี้:

### **Plan สำหรับการ Tune และเปรียบเทียบ**

- **1. Tune Q-Learning (QL):**
  - ทดลองเปลี่ยนค่า `learning_rate` และ `epsilon_decay` (Min, Max, Mid) ทีละ 5K episode

- **2. ทดลองโมเดลอื่น (ใช้พารามิเตอร์จาก QL):**
  - **Double Q-Learning (DQL):** 5K episode (ใช้ค่าเดิม และปรับลด `learning_rate`)
  - **SARSA:** 5K episode (ใช้ค่าเดิม และปรับลด `epsilon_decay`)
  - **Monte Carlo (MC):** 5K episode (ปรับค่า `discount` ให้สูงขึ้น เช่น 0.95 หรือ 1)


สรุปค่า parameter ของแต่ละ run (Run Parameters Summary) จะประกอบด้วย:

 **Run 1: QL - ค่า LR ต่ำและ epsilon decay ต่ำ**

 **Run 2: QL - ค่า LR สูงและ epsilon decay สูง**

 **Run 3: QL - ทดสอบขนาดของ state/action space**

 **Run 4: QL - ค่าเบสสำหรับการเปรียบเทียบ**

และต่อด้วยการเปรียบเทียบโมเดลอื่น ๆ (DQL, SARSA, MC) ตามที่วางแผนไว้



---

### **1. Q-Learning Tuning (Run 1-3)**  
ในการทดลองปรับ hyperparameter ของ Q-Learning พบว่า:

- **Run 1 (Min LR & Min Decay)**  
  การเรียนรู้ค่อนข้างช้า agent ใช้เวลานานกว่าจะเริ่มได้คะแนนดี เนื่องจาก learning rate ต่ำเกินไป ทำให้ update ทีละน้อย และ epsilon decay ช้า agent เลยยังสุ่มนานเกินจำเป็น  
  > ผลคือ agent ยังไม่ converge ภายใน 5,000 episodes

- **Run 2 (Max LR & Max Decay)**  
  มีการเรียนรู้เร็วขึ้นในช่วงแรก แต่กรณีนี้ค่า epsilon ลดลงเร็วมากเกินไป agent หยุดสุ่มเร็ว ทำให้ติด local optimum  
  > agent เลือกสิ่งที่ดี "ในตอนนั้น" ไวเกินไป

- **Run 3 (Lower Res in Action/Obs Space)**  
  ลดความละเอียดลง ส่งผลให้ agent มีข้อมูล input ที่ง่ายขึ้น (state space น้อยลง) และ action น้อยลง  
  > agent เรียนรู้เร็วกว่าเดิม แต่อาจควบคุมได้ไม่ละเอียดเท่าค่าพื้นฐาน

**Insight:**  
ค่า base case เดิม (Run 4) จึงเป็นจุดสมดุลระหว่างความเร็วในการเรียนรู้และความสามารถในการควบคุม ซึ่งได้ผลลัพธ์ที่ดีที่สุดในเชิง performance และ stability

---

###  **2. Final Q-Learning vs Other Models (Run 4-8)**  
เมื่อเปรียบเทียบ Q-Learning กับ DQL, SARSA และ MC ที่ใช้พารามิเตอร์เดียวกัน:

- **Q-Learning (Run 4)**  
  มีความสามารถในการเรียนรู้เร็วและเสถียร เหมาะสำหรับ environment แบบ deterministic อย่าง CartPole  

- **DQL (Run 5)**  
  ประสิทธิภาพใกล้เคียง QL หรืออาจดีกว่าเล็กน้อยในบางช่วง เพราะลด bias จาก max Q ใน Q-table ด้วยการใช้ Q สองตัว  

- **SARSA (Run 8)**  
  ระมัดระวังมากกว่า QL เพราะอิง action จริงที่จะเกิดในอนาคต  
  > ผลคือเรียนรู้ช้ากว่า QL แต่มีแนวโน้มเล่นปลอดภัยกว่า  

- **MC (Run 7)**  
  ค่อนข้างช้าหรือไม่เสถียร เนื่องจากต้องรอ episode จบก่อนถึงจะ update ทำให้ข้อมูล update ช้า  
  > เหมาะกับ environment ที่ episode สั้นและ variance ไม่เยอะเท่านั้น

---

###  **3. Tweaked Runs (Run 6 & 9)**  
การทดลองปรับพารามิเตอร์เพิ่มเติมในโมเดลอื่น:

- **Run 6: DQL with Lower LR**  
  ปรับ learning rate ให้ต่ำลง ทำให้การอัปเดต Q มีความละเอียดมากขึ้น  
  > ส่งผลให้การเรียนรู้เสถียรขึ้น แต่น่าจะช้ากว่า base case เล็กน้อย

- **Run 9: SARSA with Lower Epsilon Decay**  
  ชะลอการลด epsilon เพื่อให้ agent explore ได้นานขึ้น  
  > อาจทำให้เรียนรู้แนวทางที่ดีกว่า แต่ก็เสี่ยงต่อการเรียนรู้ช้าและไม่ converge ภายใน 5K episodes

---



#### **ตัวอย่างโค้ดสรุป Parameter สำหรับ Run ตัวอย่าง**


In [ ]:
# Parameter สำหรับ Run 4 (QL เบส)
num_of_action = 5
action_range = [-12, 12]
discretize_state_weight = [4, 8, 4, 4]
learning_rate = 0.03
n_episodes = 5000
start_epsilon = 1.0
epsilon_decay = 0.00003
final_epsilon = 0.05
discount = 1

# Parameter สำหรับการ clipping ของ state
pose_cart_bound = 3
pose_pole_bound = float(np.deg2rad(24.0))
vel_cart_bound = 15
vel_pole_bound = 15

### **Summary ของ Part 2**

- ในส่วนนี้ เราได้อธิบายถึงการกำหนดค่าพารามิเตอร์เบื้องต้นสำหรับการ Train และ Play Agent ในการ Stabilizing Cart-Pole
- ได้สรุปแนวทางการทดลองทั้งในระดับการปรับค่า (tuning) และเปรียบเทียบประสิทธิภาพของโมเดลต่าง ๆ
- มีการคำนวณขนาด state space และ Q-table เพื่อประเมินความครอบคลุมของการเยี่ยมชม state
- ในส่วน Play เราใช้คำสั่ง terminal เพื่อรัน script ทดสอบ ซึ่งเป็นส่วนที่สามารถปรับปรุงได้ตามที่อาจารย์กำหนด

ส่วนนี้เป็นพื้นฐานสำคัญที่จะนำไปสู่การวิเคราะห์ผล (Evaluation) ใน Part 3

---

# **Part 3: Evaluate `Cart-Pole` Agent Performance.**

In [ ]:
from evaluate import *

การวิเคราะห์ Q‑table นั้นมีความท้าทาย เนื่องจาก state space มีมิติสูง ในกราฟ 3D แกน z มักจะแสดงค่า maximum action‑value ของแต่ละ state ซึ่งบ่งบอกถึงคุณภาพ (expected return) ของ state นั้น ๆ

## **Baseline Experiment**
ในส่วนนี้ เราทำการประเมินผล performance ของทุก algorithm โดยใช้การทดลอง base‑line ที่ตั้งค่าพารามิเตอร์เหมือนกัน (ตาม Part 2)

### **Q-Learning**
กำหนดไฟล์ reward และ Q‑value ที่บันทึกไว้:

In [ ]:
q_learn_reward_file = "reward_value/QL_r_0.json"
q_learn_q_value_file_1000 = "q_value/Stabilize/Q_Learning/Q_Learning0/Q_Learning_0_1000_5_12_4_8.json"
q_learn_q_value_file_3000 = "q_value/Stabilize/Q_Learning/Q_Learning0/Q_Learning_0_3000_5_12_4_8.json"
q_learn_q_value_file_5000 = "q_value/Stabilize/Q_Learning/Q_Learning0/Q_Learning_0_5000_5_12_4_8.json"

plot_reward(q_learn_reward_file)
plot_reward_grouped(q_learn_reward_file, 100)

plot_q_values_3d_dual(load_q_values(q_learn_q_value_file_1000), (0,1), (4,8), (2,3), (4,4))
plot_q_values_3d_dual(load_q_values(q_learn_q_value_file_3000), (0,1), (4,8), (2,3), (4,4))
plot_q_values_3d_dual(load_q_values(q_learn_q_value_file_5000), (0,1), (4,8), (2,3), (4,4))

**Learning Efficiency (Q-Learning):**
- ในช่วง 1000 episode, agent เริ่มอัปเดต Q‑value เมื่อ cart และ pole อยู่ที่ศูนย์กลาง และเรียนรู้ว่าการเคลื่อนที่ของ cart ส่งผลต่อการสวิงของ pole
- เมื่อฝึกไปถึง 5000 episode พบว่า agent เริ่มรู้จักกับ state ที่มี reward สูง แต่ยังมีปัญหาในการควบคุม cart ให้กลับมาที่ศูนย์กลาง

**Deployment Performance (Q-Learning):**
- Agent สามารถคง pole ได้ในช่วงสั้น ๆ แต่ยังไม่สามารถควบคุมให้ cart กลับมาที่ศูนย์กลางอย่างต่อเนื่อง

### **Double Q-Learning**
กำหนดไฟล์ reward และ Q‑value:

In [ ]:
dq_learn_reward_file = "reward_value/Double_Q_Learning_r_0.json"
dq_learn_q_value_file_1000 = "q_value/Stabilize/Double_Q_Learning/Double_Q_Learning0/Double_Q_Learning_0_1000_5_12_4_8.json"
dq_learn_q_value_file_3000 = "q_value/Stabilize/Double_Q_Learning/Double_Q_Learning0/Double_Q_Learning_0_3000_5_12_4_8.json"
dq_learn_q_value_file_5000 = "q_value/Stabilize/Double_Q_Learning/Double_Q_Learning0/Double_Q_Learning_0_5000_5_12_4_8.json"

plot_reward_grouped(dq_learn_reward_file, 100)
plot_q_values_3d_dual(load_q_values(dq_learn_q_value_file_1000), (0,1), (4,8), (2,3), (4,4))
plot_q_values_3d_dual(load_q_values(dq_learn_q_value_file_3000), (0,1), (4,8), (2,3), (4,4))
plot_q_values_3d_dual(load_q_values(dq_learn_q_value_file_5000), (0,1), (4,8), (2,3), (4,4))

**Learning Efficiency (Double Q-Learning):**
- ในช่วงแรก agent มีการปรับปรุง Q‑value ในบาง state แต่ยังคงมีข้อจำกัดในด้านการประเมินค่า โดยเฉพาะในพื้นที่ที่มีความเสี่ยงต่อการ termination
**Deployment Performance (Double Q-Learning):**
- ผลการ deploy แสดงให้เห็นว่า Double Q‑Learning ลด bias ในการประเมินค่าสูงเกินไปได้บ้าง แต่ยังคงมีปัญหาในการควบคุม cart

### **SARSA**
กำหนดไฟล์ reward และ Q‑value:

In [ ]:
SARSA_reward_file = "reward_value/SARSA_r_0.json"
SARSA_q_value_file_1000 = "q_value/Stabilize/SARSA/SARSA0/SARSA_0_1000_5_12_4_8.json"
SARSA_q_value_file_3000 = "q_value/Stabilize/SARSA/SARSA0/SARSA_0_3000_5_12_4_8.json"
SARSA_q_value_file_5000 = "q_value/Stabilize/SARSA/SARSA0/SARSA_0_5000_5_12_4_8.json"

plot_reward_grouped(SARSA_reward_file, 100)
plot_q_values_3d_dual(load_q_values(SARSA_q_value_file_1000), (0,1), (4,8), (2,3), (4,4))
plot_q_values_3d_dual(load_q_values(SARSA_q_value_file_3000), (0,1), (4,8), (2,3), (4,4))
plot_q_values_3d_dual(load_q_values(SARSA_q_value_file_5000), (0,1), (4,8), (2,3), (4,4))

**Learning Efficiency (SARSA):**
- SARSA มีแนวโน้มเพิ่ม reward อย่างต่อเนื่อง แต่ Q‑value ในบริเวณขอบเขต termination ยังมีปัญหา
**Deployment Performance (SARSA):**
- Agent ที่ฝึกด้วย SARSA พบปัญหาในการควบคุมทิศทางของ cart เมื่อ pole สวิงออกนอกขอบเขต

### **Monte Carlo (MC)**
กำหนดไฟล์ reward และ Q‑value:

In [ ]:
MC_reward_file = "reward_value/MC_r_0.json"
MC_q_value_file_1000 = "q_value/Stabilize/MC/MC0/MC_0_1000_5_12_4_8.json"
MC_q_value_file_3000 = "q_value/Stabilize/MC/MC0/MC_0_3000_5_12_4_8.json"
MC_q_value_file_5000 = "q_value/Stabilize/MC/MC0/MC_0_5000_5_12_4_8.json"

plot_reward_grouped(MC_reward_file, 100)
plot_q_values_3d_dual(load_q_values(MC_q_value_file_1000), (0,1), (4,8), (2,3), (4,4))
plot_q_values_3d_dual(load_q_values(MC_q_value_file_3000), (0,1), (4,8), (2,3), (4,4))
plot_q_values_3d_dual(load_q_values(MC_q_value_file_5000), (0,1), (4,8), (2,3), (4,4))

**Learning Efficiency (MC):**
- MC มี variance สูง ทำให้ Q‑value มีความแปรปรวนมาก โดยเฉพาะเมื่อ discount factor มีค่าสูง
**Deployment Performance (MC):**
- Agent ที่ฝึกด้วย MC ยังมีปัญหาในการควบคุม cart ให้กลับมาที่ศูนย์กลาง

## **Additional Experiments**

### **Agent Saturation Check**
ตรวจสอบว่า reward over episode ยังไม่ converge เพื่อดูว่าการฝึก 5000 episode ยังไม่เพียงพอสำหรับการเรียนรู้

### **Discount Factor Effect**
ทดลองเปลี่ยนค่า Discount Factor จาก 1 ไปเป็น 0.25 และ 0.1 เพื่อดูผลต่อการอัปเดตในวิธี MC

In [ ]:
MC0_reward_file = "reward_value/MC_r_0.json"
MC1_reward_file = "reward_value/MC_r_1.json"
MC2_reward_file = "reward_value/MC_r_2.json"

plot_multiple_reward_files([MC0_reward_file, MC1_reward_file, MC2_reward_file], 100)

MC1_q_value_file_1000 = "q_value/Stabilize/MC/MC1/MC_1_1000_5_12_4_8.json"
MC1_q_value_file_3000 = "q_value/Stabilize/MC/MC1/MC_1_3000_5_12_4_8.json"
MC1_q_value_file_5000 = "q_value/Stabilize/MC/MC1/MC_1_5000_5_12_4_8.json"

plot_q_values_3d_dual(load_q_values(MC1_q_value_file_1000), (0,1), (4,8), (2,3), (4,4))
plot_q_values_3d_dual(load_q_values(MC1_q_value_file_3000), (0,1), (4,8), (2,3), (4,4))
plot_q_values_3d_dual(load_q_values(MC1_q_value_file_5000), (0,1), (4,8), (2,3), (4,4))

### **Q-Learning with Larger Space**
ทดลองเพิ่ม resolution ของ state และ action space เพื่อดูผลต่อประสิทธิภาพของ agent

In [ ]:
ql_mod_q_file_1000 = "q_value/Stabilize/Q_Learning/Q_Learning1/Q_Learning_1_1000_11_25_7_12.json"
ql_mod_q_file_3000 = "q_value/Stabilize/Q_Learning/Q_Learning1/Q_Learning_1_3000_11_25_7_12.json"
ql_mod_q_file_5000 = "q_value/Stabilize/Q_Learning/Q_Learning1/Q_Learning_1_5000_11_25_7_12.json"
ql_mod_q_file_10000 = "q_value/Stabilize/Q_Learning/Q_Learning1/Q_Learning_1_10000_11_25_7_12.json"
ql_mod_q_file_15000 = "q_value/Stabilize/Q_Learning/Q_Learning1/Q_Learning_1_15000_11_25_7_12.json"
ql_mod_q_file_20000 = "q_value/Stabilize/Q_Learning/Q_Learning1/Q_Learning_1_20000_11_25_7_12.json"
ql_mod_q_file = [ql_mod_q_file_1000, ql_mod_q_file_3000, ql_mod_q_file_5000, ql_mod_q_file_10000, ql_mod_q_file_15000, ql_mod_q_file_20000]
ql_mod_episode_labels = [f"Episode {ep}" for ep in [1000, 3000, 5000, 10000, 15000, 20000]]

plot_q_values_heatmaps_multiple_files(ql_mod_q_file, episodes_list=ql_mod_episode_labels, left_pair=(0,1), left_range=(7,12), right_pair=(2,3), right_range=(5,5))

plot_q_values_3d_dual(load_q_values(ql_mod_q_file_20000), (0,1), (7,12), (2,3), (5,5))

## **Comparisons**
### เปรียบเทียบผล reward ระหว่าง Double Q-Learning และ Q-Learning

In [ ]:
plot_multiple_reward_files(["reward_value/QL_r_1.json", "reward_value/Double_Q_learning_r_1.json"], 100)

### **Learning Rate Comparison**
เปรียบเทียบค่า learning rate 0.03 กับ 0.1 ในการเรียนรู้

**Learning Objectives:**
- เข้าใจว่าการเรียนรู้ของ agent ทำงานอย่างไรในสภาวะที่ dynamic model ไม่ทราบ
- วิเคราะห์ข้อดีข้อเสียของแต่ละ RL algorithm (Monte Carlo, SARSA, Q-Learning, Double Q-Learning)
- ตรวจสอบผลกระทบของ resolution ของ state และ action space ต่อกระบวนการเรียนรู้

**Compare Lower Space and Higher Space:**
- State/action space ที่ต่ำช่วยให้ agent เรียนรู้ได้ง่ายขึ้นเนื่องจากมี state-action pair น้อยกว่า แต่ข้อจำกัดคือการควบคุมอาจไม่ละเอียด
- ในทางกลับกัน state/action space ที่สูงช่วยให้ agent เข้าใจ environment ได้ละเอียดขึ้น แต่ต้องใช้ episode ในการเรียนรู้มากขึ้น
- การเลือก resolution ที่เหมาะสมจึงเป็นการ trade-off ระหว่างความเร็วในการเรียนรู้และความสามารถในการควบคุม

## **Question:**
### 1. **Which algorithm performs best?**
   - ผลการทดลองเบื้องต้นแสดงให้เห็นว่า ...
### 2. **Why does it perform better than the others?**
   - เนื่องจาก ...
### 3. **How do the resolutions of the action space and observation space affect the learning process? Why?**
   - การใช้ resolution ที่สูงช่วยให้ agent ได้ข้อมูลที่ละเอียดขึ้น แต่ก็ต้องอัปเดต state-action pair จำนวนมากขึ้น ซึ่งส่งผลให้ต้องใช้ episode ในการเรียนรู้มากขึ้น